# Subscription Data Exploration

A data anlystics project submitted on [mindsumo](https://www.mindsumo.com/challenges) in response to a company challenge.  

---

The purpose of this project is to explore an anonymized, subscription dataset and attempt to classify subscribtion types & analyze revenue.  The below notebook exmamines the different subscription types, classifies each subscriber, and explores basic metrics for each grouping.  It then looks at general revenue trends for the overall company, and disects the dataset to identify the most profitable subscription types.  Finally, it attempts to project future revenues and provide actionable insights for the marketing team.

Note: the subscription labels provided by the company include "one-off", "daily", "monthly", "yearly".  It is assumed that these are the only types for classification.

### Initial Project Setup

In [21]:
# I will be using plotly to create interactive visualizations throughout this notebook. 
# Here I simply install the plotting library since it is not included in the stdlib or
# the anaconda distribution of python.
!pip install plotly

In [151]:
import numpy as np
import pandas as pd
import scipy as sp
import plotly.offline as py
import plotly.graph_objs as go
from plotly import tools
py.init_notebook_mode(connected=True) # enable inline plotting for plotly

### Brief exploration of the dataset 

In [141]:
# Load dataset
transactions_df = pd.read_csv("subscription_report.csv")
transactions_df.head()

,Id,Subscription ID,Amount (USD),Transaction Date
0,1235,15447,1900,01/01/1966
1,1236,30674,7280,01/01/1966
2,1237,5293,3260,01/01/1966
3,1238,33782,4060,01/02/1966
4,1239,56714,6370,01/02/1966


In [142]:
# Rename column headers and convert dates to datetime objects for easier manipulation of data
# ... converting the dates takes a little while
transactions_df.columns = ['transaction_id', 'subscription_id', 'transaction_amt', 'transaction_date']
# transactions_df.transaction_date = pd.to_datetime(transactions_df.transaction_date, dayfirst=True)
transactions_df.head()

,transaction_id,subscription_id,transaction_amt,transaction_date
0,1235,15447,1900,01/01/1966
1,1236,30674,7280,01/01/1966
2,1237,5293,3260,01/01/1966
3,1238,33782,4060,01/02/1966
4,1239,56714,6370,01/02/1966


In [143]:
# Brief exploration of the dataset
transactions_df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
transaction_id,779999.0,391234.000000,225166.460645,1235.0,196234.5,391234.0,586233.5,781233.0
subscription_id,779999.0,30634.908547,16008.741517,3159.0,16720.0,30478.0,44497.0,58376.0
transaction_amt,779999.0,3695.845713,2084.638894,100.0,1880.0,3680.0,5500.0,7300.0


In [144]:
# A quick survey of the number of unique transactions and subscriptions
print "Number of unique transactions: {}".format(len(transactions_df.transaction_id.unique()))
print "Number of unique subscriptions: {}".format(len(transactions_df.subscription_id.unique()))
print "Number of unique transaction amounts: {}".format(len(transactions_df.transaction_amt.unique()))
# A quick check for duplicate transactions
assert len(transactions_df.transaction_id.unique()) == len(transactions_df.transaction_id)

Number of unique transactions: 779999
Number of unique subscriptions: 27609
Number of unique transaction amounts: 721


##### Summary of the Preliminary Exploration and Checks
The subscription range and the number of unique transaction amounts both seem large, as it would be assumed that standard subscription rates would be applied over the different groupings (e.g. daily, monthly, yearly, etc...).  As the data set was reported as clean and no duplicate transactions or other oddities were detected, it can be assumed that these values are correct, and the subscription scheme is highly customizeable.

### Categorization of Subscription Types

In [145]:
'''The following scripts use subscription data from 'subscription_report.csv' 
sampled above to determine the subscription type and duration for each registered 
subscriber from Jan-1966 to Dec-2014.

The subscription data is grouped according to subscription id, and the 
subscription type and duration are evaluated based on the transaction dates recorded
for each unique subscription id.  Subscription duration is reported based on the 
subscription type (i.e. years for yearly subscriptions, months for monthly subscripion, 
and days for daily subscripions).
'''
from collections import defaultdict
from datetime import datetime

def get_transactions_by_id(transaction_data):
    '''Return dictionary of subscription transaction dates for each id in data file.'''

    # initialize an empty dictionary to store an inidividual subscriber's transaction data
    subscription_transactions_by_id = defaultdict(lambda: defaultdict(list))

    # store the date and amt of every transaction recorded for each subscriber
    for row in transaction_data.itertuples():
        subscription_id = row.subscription_id
        transaction_date = pd.to_datetime(row.transaction_date, dayfirst=True)
        subscription_transactions_by_id[subscription_id]["transaction_dates_list"].append(transaction_date)
        subscription_transactions_by_id[subscription_id]["transaction_amts_list"].append(row.transaction_amt)
    return subscription_transactions_by_id

# subscription_transactions_by_id = get_transactions_by_id(transactions_df)

In [146]:
def get_type_and_duration_by_id(transaction_data):
    '''Determine the subscription type and duration for each subscription id,
    add the results to a new dict including the transaction data, and return
    the new dict.
    '''

    # load dictionary of subscription transactions for each subscripion id
    subscription_transactions_by_id = get_transactions_by_id(transaction_data)

    # initialize empty dictionary for storing subscription type and duration with
    #   the transaction data for that id
    subscription_data_by_id = {}
    
    # initialize dictionairy for storing ids of each membership type
    subscriptions_by_type = defaultdict(list)

    # loop through each subscription_id to evaluate type and duration based on transaction dates
    for sub_id in subscription_transactions_by_id:
        # check for subscription type 'one-off'
        # -- this type should have only one transaction
        # -- set duration to 1 day for type 'one-off'
        if len(subscription_transactions_by_id[sub_id]["transaction_dates_list"]) == 1:
            subscription_type = 'one-off'
            subscription_duration = 1
        # check for other subscription types
        else:
            # determine transaction year for first and second transactions
            transaction_year_one = subscription_transactions_by_id[sub_id]["transaction_dates_list"][0].year
            transaction_year_two = subscription_transactions_by_id[sub_id]["transaction_dates_list"][1].year

            # determine transaction month for first and second transactions
            transaction_month_one = subscription_transactions_by_id[sub_id]["transaction_dates_list"][0].month
            transaction_month_two = subscription_transactions_by_id[sub_id]["transaction_dates_list"][1].month

            # check for transaction type 'yearly'
            # -- this type should contain consective transaction dates with different year values
            # -- boundary case ==> "monthly" starting in december will have different year
            # -- set duration to the number of years between the first and last recorded transactions
            if transaction_year_one != transaction_year_two and \
                                            transaction_month_one + 1 != transaction_month_two:
                subscription_type = 'yearly'
                subscription_duration = len(subscription_transactions_by_id[sub_id]["transaction_dates_list"])
            # check for transaction type 'monthly'
            # -- this type should contain consective transaction dates with different month values
            # -- set duration to the number of months between the first and last recorded transactions
            elif transaction_month_one != transaction_month_two:
                subscription_type = 'monthly'
                subscription_duration = len(subscription_transactions_by_id[sub_id]["transaction_dates_list"])
            # check for transaction type 'daily'
            # -- return type 'daily' if conditions for the other types are not met
            # -- set duration to the number of days between the first and last recorded transactions
            else:
                subscription_type = 'daily'
                subscription_duration = len(subscription_transactions_by_id[sub_id]["transaction_dates_list"])
        
        # bin subscriptions by type
        subscriptions_by_type[subscription_type].append(sub_id)
        
        # store all subscription data with type and duration fields computed in a single dict
        subscription_data_by_id[sub_id] = defaultdict(dict)
        subscription_data_by_id[sub_id]["subscription_type"] = subscription_type
        subscription_data_by_id[sub_id]["subscription_duration"] = subscription_duration
        subscription_data_by_id[sub_id]["transaction_dates_list"] = \
                                            subscription_transactions_by_id[sub_id]["transaction_dates_list"]
        subscription_data_by_id[sub_id]["transaction_amts_list"] = \
                                            subscription_transactions_by_id[sub_id]["transaction_amts_list"]

    return subscriptions_by_type, subscription_data_by_id

In [147]:
# Compute subscription types and durations
subscriptions_by_type, subscription_data_by_id = get_type_and_duration_by_id(transactions_df)

In [167]:
# Visualize some data
labels = subscriptions_by_type.keys()
values = [len(subscriptions_by_type[key]) for key in subscriptions_by_type.keys()]

bar_trace = go.Bar(x=labels, y=values)

py.iplot([bar_trace], filename='subtype-distributions')

In [168]:
trace = go.Pie(labels=labels, values=values)

py.iplot([trace], filename='basic_pie_chart')

In [126]:
# Inspect a sample output from the functions above
subscription_data_by_id[56714]

defaultdict(dict,
            {'subscription_duration': 13,
             'subscription_type': 'daily',
             'transaction_amts_list': [6370,
              6370,
              6370,
              6370,
              6370,
              6370,
              6370,
              6370,
              6370,
              6370,
              6370,
              6370,
              6370],
             'transaction_dates_list': [Timestamp('1966-02-01 00:00:00'),
              Timestamp('1966-02-02 00:00:00'),
              Timestamp('1966-02-03 00:00:00'),
              Timestamp('1966-02-04 00:00:00'),
              Timestamp('1966-02-05 00:00:00'),
              Timestamp('1966-02-06 00:00:00'),
              Timestamp('1966-02-07 00:00:00'),
              Timestamp('1966-02-08 00:00:00'),
              Timestamp('1966-02-09 00:00:00'),
              Timestamp('1966-02-10 00:00:00'),
              Timestamp('1966-02-11 00:00:00'),
              Timestamp('1966-02-12 00:00:00'),
         

In [ ]:
# Visualize some data
labels = subscriptions_by_type.keys()

bar_trace = go.Bar(x=labels, y=values)

py.iplot([bar_trace], filename='subtype-distributions')